<a href="https://colab.research.google.com/github/aryan1429/deep-learning-lab/blob/main/cnn_pretrained_ants%26bees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms,models
from torch.utils.data import DataLoader

In [7]:
#Define Transformations
transforms = {'train' : transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]),

    'val' : transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
}

In [28]:
#Load the data
data_dir='/content/dataset/hymenoptera_data'


In [29]:
image_datasets = {}
image_datasets['train'] = datasets.ImageFolder(os.path.join(data_dir,'train'),transform = transforms['train'])
image_datasets['val'] = datasets.ImageFolder(os.path.join(data_dir,'val'),transform = transforms['val'])

In [30]:
data_loader = {}
data_loader['train'] = DataLoader(image_datasets['train'],batch_size = 32,shuffle = True)
data_loader['val'] = DataLoader(image_datasets['val'], batch_size = 32,shuffle = True)

In [39]:
'''model=[models.resnet50(pretrained=True),models.resnet101(pretrained=True),models.resnet152(pretrained=True),
      models.vgg16(pretrained=True),models.vgg19(pretrained=True),
      models.densenet121(pretrained=True),models.densenet169(pretrained=True),models.densenet201(pretrained=True)]
for m in model:
    model_ftrs=m.fc.in_features
    m.fc=nn.Linear(model_ftrs,out_features=2)
    criterian=nn.CrossEntropyLoss()
    optimizer=optim.Adam(m.fc.parameters(),lr=0.0001)
    print(m)'''
model = models.resnet50(pretrained = True)

model_ftrs = model.fc.in_features
model.fc = nn.Linear(model_ftrs,2)
criterian = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(),lr = 0.0001)
print(model)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 116MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [40]:
for epoch in range(10):
  for input,label in data_loader['train']:
    output = model(input)
    loss = criterian(output,label)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  print(loss.item())

0.5227653384208679
0.5211895704269409
0.5752923488616943
0.5288556814193726
0.44948285818099976
0.4409662187099457
0.3605183959007263
0.3832235038280487
0.4552121162414551
0.42344051599502563


In [41]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
  for input,label in data_loader['val']:
    output = model(input)
    _,predicted = torch.max(output,1)
    total += label.size(0)
    correct += (predicted == label).sum().item()
  print(f"Accuracy : {(correct/total)*100}%")
print(total,correct)

Accuracy : 84.9673202614379%
153 130
